In [8]:
from ipywidgets import *
import plotly.graph_objects as go
import numpy as np
import numpy as np
import scipy.fft as fft 
import scipy.signal as signal
import matplotlib.pyplot as plt
from math import *
import random as r
##############################################################
delta_w = 50
T = 1
h = 0.0001
t_len = int(T/h)
ts = np.linspace(0,T,t_len)
################################################################

SEED = r.randrange(1, 100)

checkbox = widgets.Checkbox(
    value=False,
    description='set seed',
    disabled=False,
    indent=False,
    layout=Layout(width='100px',margin='0 0 0 30px')
)

slider = widgets.IntSlider(
    value=0,
    min=0,
    max=1,
    step=1,
)


seed_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=10,
    step=1,
    description='seed:',
    layout=Layout(width='300px')
)


slider_T =  widgets.IntSlider(
    value=0,
    min=0,
    max=10,
    step=1,
    description='param:'
)

fig1 = go.Figure( layout=go.Layout(
       font=dict(family="Courier New, monospace",size=18,color="#7f7f7f"),width=600, height=600,margin=dict(l=0, r=0, t=0, b=0) ,
    legend=dict(x=0, y=1)))


In [9]:
c_x=0;
c_h=0;
c_y=0;
c_y_n=0

def BFG1():
    
    global c_x
    c_x = np.exp((0 + 1j) * ( - delta_w * ts + 2.0 / T * delta_w * ts *ts))
    c_x_real = np.real(c_x)
    c_x_imag = np.imag(c_x)
    XX=[]
    for i in range(0,t_len):
        XX.append(i)
    
    return[[XX,c_x_real],[XX,c_x_imag]]
    
    
def BFG2():
    
    global c_h
    c_h = np.exp((0 - 1j) * ( - delta_w * (T - ts) + 2.0 / T * delta_w * (T - ts) * (T - ts)))
    c_h_real = np.real(c_h)
    c_h_imag = np.imag(c_h)
    
    XX = []
    for i in range(0,t_len):
        XX.append(i)
    return [[XX,c_h_real],[XX,c_h_imag]]

def BFG3():
    global c_h,c_x,c_y
    c_y = 0.5  * signal.convolve(c_h, c_x)
    c_y_real = np.real(c_y)
    c_y_imag = np.imag(c_y)
      
    XX = []
    for i in range(0,c_y.size):
        XX.append(i)
        
    return [[XX,c_y_real],[XX,c_y_imag]]


def BFG4():
    global c_y
    c_y_abs=np.abs(c_y)
    XX = []
    for i in range(0,c_y.size):
        XX.append(i)
    return [[XX,c_y_abs]]


def BFG5():
    global c_h,c_h,c_y_n
    c_x_1 = np.hstack((c_x, np.zeros(t_len)))
    c_h_1 = np.hstack((c_h, np.zeros(t_len)))
    rc_x_1 = np.hstack((c_x, np.zeros(t_len)))
    c_h_1 = np.hstack((c_h, np.zeros(t_len)))
    c_x_2 = fft.fft(c_x_1)
    c_h_2 = fft.fft(c_h_1)
    c_y_fft = 0.5* c_x_2 *c_h_2
    c_y_n =  fft.ifft(c_y_fft)
    XX = []
    for i in range(0,len(c_y_n)):
        XX.append(i)
    return[[XX,np.abs(c_y_n)]]

def BFG6():
    global c_y_n
    delta = T / (t_len)
    ts = np.arange(0, 2*T, delta)
    w0 = 1000
    y = np.real(c_y_n * np.exp((0 + 1j) * w0 * ts))
    XX=[]
    for i in range(0,y.size):
            XX.append(i)
    return [[XX,y]]





def css_widget(text, font_size=10):
    html = "<h1 style='font-size:"+str(font_size)+"px; margin-left: 50px; margin-top: 0;margin-bottom:0;width:350px'>" + str(text) + "</h1>"
    W = widgets.HTML(value=html)
    return W


In [11]:

step_btn = widgets.Button(description='refresh', button_style='primary')
reset_btn = widgets.Button(description='reset', button_style='primary')

def custom_layout(w,h):

    ll=Layout(width=str(w) + 'px', height=str(h) + 'px', border='solid')

    return ll 


def showText(text):
    print("\t", text)





def function(widget, figure, fun_for_data):
    
    global SEED
    
    figure.data = []   
    
    if(checkbox.value == True):
        np.random.seed(seed_slider.value)
    else:
        np.random.seed(SEED)
 
    data = fun_for_data() 
    
    for values in data:
        figure.add_trace(go.Scatter(x=values[0], y=values[1]))
        
        

        


    figure.show()


def custom_display():
    
    text=""" На цифровой сигнал подается БГШ (белый гаусс. шум). Пропускаем через фильтр Баттерворта. Рассчитать отношения мощностей до и после фильтрации.
    В работе используются два фильтра: фильтр Баттерворта и встроенный в пакет scipy.signal lowess """
    
    
    HH_ui1 = VBox([interactive_plot1])
    HH_ui2 = VBox([interactive_plot2])
    HH_ui3 = VBox([interactive_plot3])
    HH_ui4 = VBox([interactive_plot4])
    HH_ui5 = VBox([interactive_plot5])
    HH_ui6 = VBox([interactive_plot6])
    
    
    tab = widgets.Tab(children = [ HH_ui1, HH_ui2,HH_ui3,HH_ui4,HH_ui5,HH_ui6 ])
    tab.set_title(0, '1')
    tab.set_title(1, '2')
    tab.set_title(2, '3')
    tab.set_title(3, '4')
    tab.set_title(4, '5')
    tab.set_title(5, '6')
    
    
    ui = VBox([slider_T, 
               HBox([checkbox, seed_slider], layout =custom_layout("auto", "auto") ), 
               HBox([step_btn,reset_btn], layout = Layout(  display='flex',
                                                            flex_flow='row',
                                                            justify_content='space-between'))
              ])
    
    
    
    tab2 = widgets.Tab(children = [css_widget("Задача 5",15), ui])
    tab2.set_title(0, 'Теория')
    tab2.set_title(1, 'Параметры')

    
    

    
    HHH_2 = (Box([tab], layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='solid',
                    width='60%')))
    
    
    
    #     Htext = VBox([css_widget("Задача 1",15),
    #                   css_widget(text,12),
    #                   css_widget(text_for_slider1,12),

    #                   out3,
    #                   css_widget(text_for_slider2,12),
    #                   out4,

    #                   css_widget(text_for_slider3,12),
    #                   out5])

    
    HHH_3 = (Box([tab2], layout=Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    border='solid',
                    width='40%')))
    
    item_2 = HBox([HHH_2,HHH_3])

    display(VBox([item_2]))
    
    toggle_visibility(None)


def toggle_visibility(button):
    seed_slider.layout.visibility = 'hidden' if (checkbox.value == False) else  'visible'


def clicker(b):
    global SEED
    SEED = r.randrange(1, 100)
    slider.value = 1 if (slider.value == 0) else  0  
    custom_display()
       
step_btn.on_click(clicker)
checkbox.observe(toggle_visibility)

interactive_plot1 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG1) })
interactive_plot2 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG2) })
interactive_plot3 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG3) })
interactive_plot4 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG4) })
interactive_plot5 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG5) })
interactive_plot6 = interactive_output(function, {'widget':slider, 'figure':fixed(fig1), 'fun_for_data': fixed(BFG6) })

custom_display()

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:50% !important; }</style>"))